============================================================
# VTI – PYTHON + SQL + RFM + KNN – CAPSTONE PROJECT
# DATASET: ONLINE RETAIL
============================================================

In [89]:
# =========================
# QUESTION 1: IMPORT LIBRARIES
# =========================
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# Visualization config
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,6)

print(">>> Import libraries thành công")

>>> Import libraries thành công


In [90]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-G3F50P50\\MSSQLSERVER03;"
    "DATABASE=VTI;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

print("✔ Kết nối SQL thành công!")

✔ Kết nối SQL thành công!


In [91]:
# =========================
# QUESTION 2: LOAD DATA FROM CSV
# =========================

# Đọc dữ liệu Online Retail
df = pd.read_csv("retail.csv", encoding="latin1")

# Chuẩn hóa tên cột
df.columns = df.columns.str.strip().str.replace(" ", "_")

print(">>> Load dữ liệu thành công")
df.head()


>>> Load dữ liệu thành công


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [92]:
# =========================
# QUESTION 3: DATA CLEANING
# =========================

# Chuyển kiểu dữ liệu
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df['Customer_ID'] = pd.to_numeric(df['Customer_ID'], errors='coerce')

# Loại dữ liệu không hợp lệ
df = df.dropna(subset=['Customer_ID'])
df = df[~df['Invoice'].astype(str).str.startswith('C')]
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

print(">>> Làm sạch dữ liệu hoàn tất")
df.head()

>>> Làm sạch dữ liệu hoàn tất


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [93]:
# =========================
# QUESTION 4: CREATE REVENUE
# =========================
df['Revenue'] = df['Quantity'] * df['Price']

df[['Quantity','Price','Revenue']].head()


,Quantity,Price,Revenue
0,12,6.95,83.4
1,12,6.75,81.0
2,12,6.75,81.0
3,48,2.10,100.8
4,24,1.25,30.0


In [94]:
# =========================
# QUESTION 5: RFM CALCULATION
# =========================

reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('Customer_ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'Invoice': 'nunique',
    'Revenue': 'sum'
}).reset_index()

rfm.columns = ['Customer_ID','R','F','M']

print(">>> Tính RFM hoàn tất")
rfm.head()


>>> Tính RFM hoàn tất


,Customer_ID,R,F,M
0,12346.0,326,12,77556.46
1,12347.0,2,8,5633.32
2,12348.0,75,5,2019.40
3,12349.0,19,4,4428.69
4,12350.0,310,1,334.40


In [95]:
# =========================
# QUESTION 6: SCALE RFM
# =========================
scaler = MinMaxScaler()

rfm[['R_scaled','F_scaled','M_scaled']] = scaler.fit_transform(
    rfm[['R','F','M']])

rfm.head()


,Customer_ID,R,F,M,R_scaled,F_scaled,M_scaled
0,12346.0,326,12,77556.46,0.440379,0.027708,0.127384
1,12347.0,2,8,5633.32,0.001355,0.017632,0.009248
2,12348.0,75,5,2019.40,0.100271,0.010076,0.003312
3,12349.0,19,4,4428.69,0.024390,0.007557,0.007269
4,12350.0,310,1,334.40,0.418699,0.000000,0.000544


In [96]:
# =========================
# QUESTION 7: WEIGHTED RFM SCORE
# =========================

rfm['R_adj'] = 1 - rfm['R_scaled']

rfm['RFM_SCORE'] = (
    rfm['R_adj'] * 0.3 +
    rfm['F_scaled'] * 0.5 +
    rfm['M_scaled'] * 0.2
)

rfm[['R_adj','F_scaled','M_scaled','RFM_SCORE']].head()


,R_adj,F_scaled,M_scaled,RFM_SCORE
0,0.559621,0.027708,0.127384,0.207217
1,0.998645,0.017632,0.009248,0.310259
2,0.899729,0.010076,0.003312,0.275619
3,0.975610,0.007557,0.007269,0.297915
4,0.581301,0.000000,0.000544,0.174499


In [97]:
# =========================
# QUESTION 8: RFM SEGMENTATION
# =========================

rfm['Label_RFM_SCORE'] = pd.cut(
    rfm['RFM_SCORE'],
    bins=[-1, 0.4, 0.7, 1.1],
    labels=['DELUX','STANDARD','VIP']
)

rfm['VIP_Label'] = (rfm['Label_RFM_SCORE'] == 'VIP').astype(int)

print(">>> Phân khúc khách hàng hoàn tất")
rfm[['RFM_SCORE','Label_RFM_SCORE','VIP_Label']].head()



>>> Phân khúc khách hàng hoàn tất


,RFM_SCORE,Label_RFM_SCORE,VIP_Label
0,0.207217,DELUX,0
1,0.310259,DELUX,0
2,0.275619,DELUX,0
3,0.297915,DELUX,0
4,0.174499,DELUX,0


In [98]:
# =========================
# QUESTION 9: DISTRIBUTION ANALYSIS
# =========================

print("--- Phân phối Nhãn VIP ---")
print(rfm['VIP_Label'].value_counts())

print(f"\nRFM Score Trung bình: {rfm['RFM_SCORE'].mean():.4f}")
print(f"RFM Score Cao nhất: {rfm['RFM_SCORE'].max():.4f}")
print(f"RFM Score Thấp nhất: {rfm['RFM_SCORE'].min():.4f}")


--- Phân phối Nhãn VIP ---
VIP_Label
0    5876
1       2
Name: count, dtype: int64

RFM Score Trung bình: 0.2262
RFM Score Cao nhất: 0.8972
RFM Score Thấp nhất: 0.0000


In [99]:
# =========================
# QUESTION 10: KNN MODEL
# =========================

X = rfm[['R_scaled','F_scaled','M_scaled']]
y = rfm['VIP_Label']

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y)

print(">>> Huấn luyện KNN thành công")


>>> Huấn luyện KNN thành công


In [100]:
# =========================
# QUESTION 11: NEW CUSTOMER PREDICTION
# =========================

new_users = pd.DataFrame({
    'R_scaled': [0.92,0.88,0.40,0.30,0.95,0.50,0.25,0.70,0.35,0.60],
    'F_scaled': [0.10,0.20,0.60,0.80,0.05,0.50,0.90,0.30,0.75,0.40],
    'M_scaled': [0.15,0.30,0.70,0.80,0.10,0.55,0.85,0.40,0.75,0.45]
})

new_users['Customer_ID'] = [f'C{i:03d}' for i in range(1,11)]
new_users['Predicted_VIP'] = knn.predict(new_users[['R_scaled','F_scaled','M_scaled']])

new_users


,R_scaled,F_scaled,M_scaled,Customer_ID,Predicted_VIP
0,0.92,0.10,0.15,C001,0
1,0.88,0.20,0.30,C002,0
2,0.40,0.60,0.70,C003,0
3,0.30,0.80,0.80,C004,0
4,0.95,0.05,0.10,C005,0
5,0.50,0.50,0.55,C006,0
6,0.25,0.90,0.85,C007,0
7,0.70,0.30,0.40,C008,0
8,0.35,0.75,0.75,C009,0
9,0.60,0.40,0.45,C010,0


In [101]:
# =========================
# QUESTION 12: SEGMENT SUMMARY
# =========================

segment_summary = rfm.groupby('Label_RFM_SCORE').agg(
    Avg_Recency=('R','mean'),
    Avg_Frequency=('F','mean'),
    Avg_Monetary=('M','mean'),
    Customer_Count=('Customer_ID','count')
).reset_index()

segment_summary['Customer_Percentage'] = (
    segment_summary['Customer_Count'] / segment_summary['Customer_Count'].sum() * 100
)

segment_summary = segment_summary.round(2)
segment_summary


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7204\21541174.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  segment_summary = rfm.groupby('Label_RFM_SCORE').agg(


,Label_RFM_SCORE,Avg_Recency,Avg_Frequency,Avg_Monetary,Customer_Count,Customer_Percentage
0,DELUX,202.27,5.67,2379.71,5850,99.52
1,STANDARD,6.38,118.27,133444.72,26,0.44
2,VIP,1.00,367.00,176286.01,2,0.03


In [102]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-G3F50P50\\MSSQLSERVER03;"
    "DATABASE=VTI;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

cursor = conn.cursor()
print("✔ Kết nối SQL Server thành công")

for _, row in rfm.iterrows():
    cursor.execute("""
        INSERT INTO RFM_Result (Customer_ID, R, F, M, RFM_SCORE, VIP_Label)
        VALUES (?, ?, ?, ?, ?, ?)
    """,
    int(row['Customer_ID']),
    float(row['R']),
    float(row['F']),
    float(row['M']),
    float(row['RFM_SCORE']),
    int(row['VIP_Label'])
    )

conn.commit()
print("✔ Đã insert dữ liệu từ Python vào SQL Server")


✔ Kết nối SQL Server thành công
✔ Đã insert dữ liệu từ Python vào SQL Server
